In [ ]:
import os as os
import sys
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, BatchNormalization,Activation
from keras.preprocessing.image import img_to_array, ImageDataGenerator, array_to_img
from keras import initializers
from keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt
from matplotlib import image
import sklearn.model_selection 

import pandas as pd
import numpy as np
from PIL import Image

In [ ]:
os.chdir(r"Galaxy Zoo")
#Read in labels
training = pd.read_csv( 'classification_labels.csv');
index = np.array(training['GalaxyID'])
training = training.set_index('GalaxyID')

In [ ]:
def prep_data(m,n,shuffle=False):
    files= os.listdir() #File names
    if shuffle==True:
        np.random.shuffle(files)
    sample= files[m:n] #Use a sample from total data
    images_array = []
    probs  = []

    for i in range(len(sample)):
        images_array.append(image.imread(sample[i])) #Read in image
        j= int(sample[i][:-4]) #Location in dataframe
        probs.append(np.array(training.loc[j])) #Corresponding label row

    images = np.array(images_array)  
    labels = np.array(probs)
    
    return images, labels

In [ ]:
#Directory where images are stored
os.chdir(r"Galaxy Zoo\Training\compress")

In [ ]:
m=0
n=7000
train_images, train_labels=prep_data(m,n)

In [ ]:
cnn=Sequential()

cnn.add(Conv2D(48, [5, 5], input_shape=(212, 212, 3), activation='relu'))
cnn.add(MaxPool2D([3, 3], [3, 3]))

cnn.add(Conv2D(96, [5, 5], activation='relu'))
cnn.add(MaxPool2D([2, 2], [2, 2]))

cnn.add(Conv2D(192, [3, 3], activation='relu'))

cnn.add(Conv2D(192, [3, 3], activation='relu'))

cnn.add(Conv2D(384, [3, 3], activation='relu'))

cnn.add(Conv2D(384, [3, 3], activation='relu'))
cnn.add(MaxPool2D([3, 3], [3, 3] ))

cnn.add(Dense(2048, activation='relu'))
cnn.add(Dropout(0.5))

cnn.add(Dense(2048, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Flatten())

cnn.add(Dense(37, activation='sigmoid'))

In [ ]:
cnn.compile(optimizer='adam', 
              loss="binary_crossentropy", 
              metrics=['accuracy'])

cnn.summary()

In [ ]:
callback = EarlyStopping(monitor='accuracy', patience=2)
cnn.fit(train_images, train_labels,batch_size=32,epochs=16,validation_split=0.25,callbacks=[callback])